In [1]:
from langchain.vectorstores import FAISS
from langchain.embeddings import GPT4AllEmbeddings
from typing import List, Dict, Tuple

vector_db_path = "vectorstores/db_faiss"

# Initialize the embedding model
embedding_model = GPT4AllEmbeddings(model_file="models/all-MiniLM-L6-v2-f16.gguf")

# Load the FAISS vector database with the embedding model
db = FAISS.load_local(folder_path=vector_db_path, embeddings=embedding_model, allow_dangerous_deserialization = True)

# Perform a similarity search
def search_query(query: str, k: int = 30):
    """
    Perform a similarity search on the vector database.
    
    :param query: The query string.
    :param k: The number of top results to return.
    :return: The top results as a list of strings.
    """
    results = db.similarity_search(query, k=k)
    return [result.page_content for result in results]

# Example query
# query = "Thủ tục cấp chứng chỉ hành nghề dược gồm các nội dung gì?"
# top_results = search_query(query, k=30)

# # Print results
# print("\nTop Results:")
# for result in top_results:
#     print(result)


Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS
from transformers import BitsAndBytesConfig

import torch

# Define the path for the model
model_file = "AITeamVN/Vi-Qwen2-7B-RAG"
# model_file = "vilm/vinallama-7b-chat"

# Define the FAISS vector store path
vector_db_path = "vectorstores/db_faiss"

# Load the Hugging Face model and tokenizer
def load_huggingface_model(model_file):
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Tải trọng số được lượng hóa trước theo định dạng 4 bit
        bnb_4bit_quant_type="nf4",  # Sử dụng loại lượng hóa "nf4" cho trọng số 4 bit
        bnb_4bit_compute_dtype=torch.bfloat16,  # Sử dụng torch.bfloat16 cho các phép tính trung gian
        bnb_4bit_use_double_quant=True,  # Sử dụng độ chính xác kép để lượng hóa kích hoạt
    )
    # quantization_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold = 6.0)
    model = AutoModelForCausalLM.from_pretrained(model_file, device_map="auto", quantization_config=quantization_config,)
    tokenizer = AutoTokenizer.from_pretrained(model_file)
    return model, tokenizer

# Read the vector database (FAISS)
def read_vectors_db():
    embedding_model = GPT4AllEmbeddings(model_file="models/all-MiniLM-L6-v2-f16.gguf")
    db = FAISS.load_local(vector_db_path, embedding_model, allow_dangerous_deserialization=True)
    return db

# Perform similarity search on the vector database
def search_vector_db(query, k=2):
    db = read_vectors_db()
    results = db.similarity_search(query, k=k)
    return [result.page_content for result in results]
# Load the model and tokenizer
model, tokenizer = load_huggingface_model(model_file)


system_prompt = "Bạn là một trợ lí Tiếng Việt nhiệt tình và trung thực. Hãy luôn trả lời một cách hữu ích nhất có thể."
template = '''Chú ý các yêu cầu sau:
- Câu trả lời phải chính xác và đầy đủ nếu ngữ cảnh có câu trả lời. 
- Chỉ sử dụng các thông tin có trong ngữ cảnh được cung cấp.
- Chỉ cần từ chối trả lời và không suy luận gì thêm nếu ngữ cảnh không có câu trả lời.
- Nếu nhiều nội dung được lấy từ cùng 1 khoản trong tài liệu đã cho, trả về toàn bộ nội dung trong khoản đó một cách chính xác nhất, không thực hiện tóm tắt lại.
Hãy trả lời câu hỏi dựa trên ngữ cảnh:
### Ngữ cảnh :
{context}

### Câu hỏi :
{question}

### Trả lời :'''





/home/phongnd/anaconda3/envs/llm2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:35<00:00,  8.95s/it]


In [3]:
question = '''Hồ sơ đề nghị cấp chứng chỉ hành nghề  dược đối với người nước ngoài gồm những nội dung nào, trả lời một cách chi tiết và đầy đủ nhất'''
context_list = search_vector_db(question, k = 50)
context = "\n".join(context_list)
conversation = [{"role": "system", "content": system_prompt }]
conversation.append({"role": "user", "content": template.format(context = context, question = question)})
with torch.inference_mode():
    text = tokenizer.apply_chat_template(
        conversation,
        tokenize=False,
        add_generation_prompt=True)
    model_inputs = tokenizer(text,return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=2048,
        temperature = 0.5,
        #top_p=0.95,
        #top_k=40,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [4]:
import textwrap

# Example text (you can replace this with your query result)
text = response

# Function to wrap text
def wrap_text(text, width=80):
    wrapper = textwrap.TextWrapper(width=width, replace_whitespace=False)
    wrapped_text = "\n".join([wrapper.fill(line) for line in text.splitlines()])
    return wrapped_text                                                                                                                                                                                            

# Wrap and print the text
wrapped_text = wrap_text(text, width=140)
print(wrapped_text)


Dựa trên ngữ cảnh cung cấp, hồ sơ đề nghị cấp Chứng chỉ hành nghề dược đối với người nước ngoài bao gồm các nội dung sau:

1. **Đơn đề nghị cấp Chứng chỉ hành nghề dược**:
   - Thực hiện theo Mẫu số 02 tại Phụ lục I ban hành kèm theo Nghị định này.
   - 02 ảnh chân dung cỡ 4 cm x 6 cm của người đề nghị cấp Chứng chỉ hành nghề dược, chụp trên nền trắng trong thời gian không quá 06 tháng.

2. **Bản sao có chứng thực văn bằng chuyên môn**:
   - Đối với các văn bằng do cơ sở đào tạo nước ngoài cấp, phải kèm theo bản sao có chứng thực giấy công nhận tương đương của cơ quan có
thẩm quyền về công nhận tương đương theo quy định tại khoản 2 Điều 18 của Nghị định này.

3. **Giấy chứng nhận sức khỏe**:
   - Bản chính hoặc bản sao có chứng thực Giấy chứng nhận sức khỏe do cơ sở khám bệnh, chữa bệnh cấp theo quy định tại Luật khám bệnh, chữa
bệnh.

4. **Bản chính hoặc bản sao chứng thực giấy xác nhận kết quả thi**:
   - Đối với trường hợp Chứng chỉ hành nghề dược cấp theo hình thức thi, phải có bản